# Stateful versus Stateless Simple Example

## Overview

There are three resource paths:
1. POST ```/current_user``` simulates login and puts a ```user``` header in responses. Callers should use the header in subsequent calls.<br><br>
2. GET ```/stateful_adder?added_value=nnn``` adds the integer ```nnn``` to the current value maintain in the per use session storage in the microservice.<br><br>
2. GET ```/stateless_adder?added_value=nnn``` adds the integer ```nnn``` to the current value maintain in the per use session storage in the microservice. The state (current_value) is returned in a header that the caller must pass in subsequent calls.

## Setup

In [1]:
import requests

In [3]:
# Set this to the root URL used for starting the service.
#
base_url = "http://0.0.0.0:5002"

# URL for setting user to simulate login.
current_user_url = "http://0.0.0.0:5002/current_user"
value_to_add=0

# Urls for the various adders.
stateful_url = f"http://0.0.0.0:5002/stateful_adder?added_value={value_to_add}"
stateless_url = f"http://0.0.0.0:5002/stateless_adder?added_value={value_to_add}"

## Stateful Client Example

In [4]:
# Set the user in the context and create session storage in server.
#
user_name = "Don_Stateful"

body = {
    "user_name": user_name
}

result = requests.post(
    current_user_url,
    json=body
)

result

<Response [201]>

In [5]:
user_name = result.headers.get("user_name", None)
user_name

'Don_Stateful'

In [11]:
# Testing adding. You can run this cell as many times as you want.
#
value_to_add = 20
stateful_url = f"http://0.0.0.0:5002/stateful_adder?added_value={value_to_add}"
headers = {"user": user_name}

print("Stateful URL = ", stateful_url)

result = requests.get(stateful_url, headers=headers)

print("Result = ", result.status_code)

if result.status_code == 200:
    print("Addition result = ", result.json())
    

Stateful URL =  http://0.0.0.0:5002/stateful_adder?added_value=20
Result =  200
Addition result =  {'new_value': 60}


## Stateless Example

In [12]:
# Set the user in the context and create session storage in server.
#
user_name = "Don_Stateless"

body = {
    "user_name": user_name
}

result = requests.post(
    current_user_url,
    json=body
)

result

<Response [201]>

In [13]:
user_name = result.headers.get("user_name", None)
user_name

'Don_Stateless'

In [14]:
# Set the user
headers = {"user": user_name}

In [16]:
# Testing adding. You can run this cell as many times as you want.
# Note that I need to pass the header back.

value_to_add = 20

stateless_url = f"http://0.0.0.0:5002/stateless_adder?added_value={value_to_add}"

print("Stateless URL = ", stateless_url)

result = requests.get(stateless_url, headers=headers)

print("Result = ", result.status_code)

if result.status_code == 200:
    print("Addition result = ", result.json())
    headers["current_value"] = result.headers.get("current_value")

Stateless URL =  http://0.0.0.0:5002/stateless_adder?added_value=20
Result =  200
Addition result =  {'new_value': 40}
